In [1]:
import re
import numpy as np
import pandas as pd

def parse_formula(formula):
    # Regular expression to match elements and their amounts
    pattern = r"([A-Z][a-z]*)([0-9./]*)"
    matches = re.findall(pattern, formula)
    
    # Dictionary to store element amounts
    element_amounts = {}
    
    for element, amount in matches:
        if amount == "":  # If no amount is specified, it's 1
            amount = 1
        elif "/" in amount:  # Handle fractions
            numerator, denominator = map(float, amount.split("/"))
            amount = numerator / denominator
        else:  # Convert to float
            amount = float(amount)
        
        if element in element_amounts:
            element_amounts[element] += amount
        else:
            element_amounts[element] = amount
    
    return element_amounts

In [2]:
formula = parse_formula("NaMn0.5Ni0.5O2")
formula

{'Na': 1, 'Mn': 0.5, 'Ni': 0.5, 'O': 2.0}

In [3]:
srp_values_df = pd.read_csv("all_elements_oxidation_states(in).csv")
srp_values_df

,Element,Ox,Red,SRP (V)
0,Sr,1,0,-4.101
1,Ca,1,0,-3.800
2,Th,4,3,-3.600
3,Pr,3,2,-3.100
4,Cs,1,0,-3.026
...,...,...,...,...
190,Kr,2,0,3.270
191,Pt,4,2,0.726
192,In,2,1,-0.400
193,In,3,2,-0.490


In [4]:
unique_elements = srp_values_df['Element'].value_counts()
static_elements = unique_elements[unique_elements == 1].index.tolist()
dynamic_elements = unique_elements[unique_elements > 1].index.tolist()

In [5]:
max_iter = 100
num_iter = 0

srp_static = 0
srp_dynamic = 0


# Loop through the formula and calculate SRP for static and dynamic elements
for element in formula:
    if element in static_elements:
        redox = srp_values_df[srp_values_df["Element"] == element]["SRP (V)"]
        srp_static += redox.values[0] * formula[element]
        
    elif element in dynamic_elements:
        redox = srp_values_df[srp_values_df["Element"] == element]["SRP (V)"]
        redox = redox.sort_values(ascending=True).iloc[0]
        srp_dynamic += redox * formula[element]
        
total_srp = round(srp_static + srp_dynamic, 3)

print("Total SRP:", total_srp)
print("Static SRP:", srp_static)
print("Dynamic SRP:", srp_dynamic)

# Find the dynamic element with the lowest SRP value
lowest_dynamic_element = min(formula.keys(), key=lambda el: srp_values_df[(srp_values_df["Element"] == el) & (el in dynamic_elements)]["SRP (V)"].min() if el in dynamic_elements else float('inf'))
print("Dynamic element with the lowest SRP value:", lowest_dynamic_element)

Total SRP: -1.203
Static SRP: -0.25
Dynamic SRP: -0.9525
Dynamic element with the lowest SRP value: Mn


In [8]:
srp_values_df[srp_values_df["Element"] == "Ni"]

,Element,Ox,Red,SRP (V)
96,Ni,2,0,-0.720
103,Ni,4,2,-0.490
117,Ni,2,0,-0.257
181,Ni,4,2,1.590


In [15]:
ChemicalData = {
    "list1": {},
    "list2": {}
}

# Populate list1 with redox data
for _, row in srp_values_df.iterrows():
    element = row["Element"]
    ox = row["Ox"]
    red = row["Red"]
    srp = row["SRP (V)"]
    if element not in ChemicalData["list1"]:
        ChemicalData["list1"][element] = []
    ChemicalData["list1"][element].append((ox, red, srp))

# Populate list2 with fixed redox states
for element in unique_elements.index:
    if element in static_elements:
        ChemicalData["list2"][element] = srp_values_df[srp_values_df["Element"] == element]["Red"].iloc[0]
    else:
        ChemicalData["list2"][element] = -1

In [16]:
ChemicalData["list1"]["Ni"]

[(2, 0, -0.72), (4, 2, -0.49), (2, 0, -0.257), (4, 2, 1.59)]

In [ ]:
import math
import copy

MAX_ITERATIONS = 100

def perform_redox_analysis(formula, include_intermediate=True, export_final=False):
    local_list1 = copy.deepcopy(ChemicalData["list1"])
    previous_charge_with_delta = None
    iteration_count = 0
    output = ""

    element_with_final_lowest_srp = None
    final_delta_ox_max = 0
    final_charge_with_delta = 0
    element_quantity = 0
    final_ox = 0
    final_red = 0

    while True:
        iteration_count += 1
        if iteration_count > MAX_ITERATIONS:
            if include_intermediate and not export_final:
                output += "[ERROR] Maximum iterations reached. Stopping analysis.\n"
            break

        sum_non_m = 0
        sum_m = 0

        if include_intermediate and not export_final:
            output += "Element Details:\n"

        for element, quantity in formula.items():
            if ChemicalData["list2"][element] != -1:
                red = ChemicalData["list2"][element]
                sum_non_m += quantity * red
                if include_intermediate and not export_final:
                    output += f"Element: {element}, Quantity: {round(quantity, 3)}, Red: {red}\n"
            else:
                redox_data = next((d for d in local_list1[element] if d[1] > 0), None)
                if redox_data:
                    ox, red, srp = redox_data
                    sum_m += quantity * red
                    if include_intermediate and not export_final:
                        output += f"Element: {element}, Quantity: {round(quantity, 3)}, Ox: {ox}, Red: {red}, SRP: {round(srp, 3)}\n"

        total_charge = round(sum_non_m + sum_m, 3)
        if include_intermediate and not export_final:
            output += f"Sum of Non-M: {round(sum_non_m, 3)}, Sum of M: {round(sum_m, 3)}, Total: {round(total_charge, 3)}\n"

        # Find M-element with lowest SRP
        m_candidates = [
            {
                "element": element,
                "quantity": quantity,
                "data": next((d for d in local_list1[element] if d[1] > 0), None)
            }
            for element, quantity in formula.items()
            if ChemicalData["list2"][element] == -1 and local_list1[element]
        ]
        m_candidates = [e for e in m_candidates if e["data"] is not None]
        m_candidates.sort(key=lambda e: e["data"][2])  # Sort by SRP

        if not m_candidates:
            if include_intermediate and not export_final:
                output += "No valid M-element found for further iterations.\n"
            break

        lowest_srp = m_candidates[0]
        element = lowest_srp["element"]
        quantity = lowest_srp["quantity"]
        ox, red, srp = lowest_srp["data"]

        delta_ox_max = round((ox - red) * quantity, 3)
        charge_with_delta = round(total_charge + delta_ox_max, 3)

        if include_intermediate and not export_final:
            output += f"M-element with lowest SRP: {element}, SRP: {round(srp, 3)}\n"
            output += f"{element} ΔOxMax = {delta_ox_max}, ΔOxMax + Total Charge = {charge_with_delta}\n"

        if charge_with_delta == previous_charge_with_delta:
            if include_intermediate and not export_final:
                output += "No change in ΔOxMax + Total Charge. Stopping.\n"
            break

        previous_charge_with_delta = charge_with_delta

        if charge_with_delta < 0:
            # Promote oxidation
            total_charge += delta_ox_max
            # Remove current redox pair
            current_red = red
            local_list1[element] = [d for d in local_list1[element] if d[1] != current_red]
            # Try to add next step (Ox → Ox) if nothing else is available
            next_redox = next((d for d in local_list1[element] if d[1] == ox), None)
            if not next_redox:
                local_list1[element].append((ox, ox, float("inf")))  # Dummy large SRP
        else:
            # Final state reached
            element_with_final_lowest_srp = element
            final_delta_ox_max = delta_ox_max
            final_charge_with_delta = charge_with_delta
            element_quantity = quantity
            final_ox = ox
            final_red = red
            break

        if include_intermediate and not export_final:
            output += "\n-----------------------------------\n\n"

    return output  # Add final charge summary and formatting as needed

# Example usage
output = perform_redox_analysis(formula, include_intermediate=True, export_final=False)
print(output)

Element Details:
Element: Na, Quantity: 1, Red: 0
Element: Mn, Quantity: 0.5, Ox: 4, Red: 3, SRP: 0.95
Element: Ni, Quantity: 0.5, Ox: 4, Red: 2, SRP: -0.49
Element: O, Quantity: 2.0, Red: -2
Sum of Non-M: -4.0, Sum of M: 2.5, Total: -1.5
M-element with lowest SRP: Ni, SRP: -0.49
Ni ΔOxMax = 1.0, ΔOxMax + Total Charge = -0.5

-----------------------------------

Element Details:
Element: Na, Quantity: 1, Red: 0
Element: Mn, Quantity: 0.5, Ox: 4, Red: 3, SRP: 0.95
Element: Ni, Quantity: 0.5, Ox: 4, Red: 4, SRP: inf
Element: O, Quantity: 2.0, Red: -2
Sum of Non-M: -4.0, Sum of M: 3.5, Total: -0.5
M-element with lowest SRP: Mn, SRP: 0.95
Mn ΔOxMax = 0.5, ΔOxMax + Total Charge = 0.0

